# Надо намайнить github, kaggle, linkedin юзернеймов

In [1]:
import pandas as pd
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, Table, Column, Integer, String, Boolean, DateTime, Date, MetaData, ForeignKey
from sqlalchemy.engine.url import URL
from sqlalchemy.orm import sessionmaker
from sqlalchemy.pool import NullPool
import csv
import re
import editdistance
import json

local_name = 'sqlite:///../ods-slack.db'
engine = create_engine(local_name, echo=True)


## Посмотрим на данные

In [2]:
users = pd.read_sql_query('select * from imported_user_data', engine)
messages = pd.read_sql_query('select * from imported_messages', engine)


2018-03-20 18:12:01,446 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-03-20 18:12:01,447 INFO sqlalchemy.engine.base.Engine ()
2018-03-20 18:12:01,448 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-03-20 18:12:01,449 INFO sqlalchemy.engine.base.Engine ()
2018-03-20 18:12:01,450 INFO sqlalchemy.engine.base.Engine select * from imported_user_data
2018-03-20 18:12:01,451 INFO sqlalchemy.engine.base.Engine ()
2018-03-20 18:12:01,653 INFO sqlalchemy.engine.base.Engine select * from imported_messages
2018-03-20 18:12:01,654 INFO sqlalchemy.engine.base.Engine ()


In [3]:
print(users.shape)
print(messages.shape)

(11159, 34)
(751861, 12)


In [4]:
users.head()

,id,name,deleted,tz,tz_label,tz_offset,is_admin,is_owner,is_primary_owner,is_restricted,...,avatar_hash,first_name,last_name,image_24,image_32,image_48,image_72,image_192,image_512,team
0,U4EMUGFNG,0,0,Asia/Jerusalem,Israel Standard Time,7200,0,0,0,0,...,g204d33bf41b,0,,https://secure.gravatar.com/avatar/204d33bf41b...,https://secure.gravatar.com/avatar/204d33bf41b...,https://secure.gravatar.com/avatar/204d33bf41b...,https://secure.gravatar.com/avatar/204d33bf41b...,https://secure.gravatar.com/avatar/204d33bf41b...,https://secure.gravatar.com/avatar/204d33bf41b...,T040HKJE3
1,U7SE7ANCW,00serd00,0,Asia/Vladivostok,Vladivostok Time,36000,0,0,0,0,...,g428cf084b50,,,https://secure.gravatar.com/avatar/428cf084b50...,https://secure.gravatar.com/avatar/428cf084b50...,https://secure.gravatar.com/avatar/428cf084b50...,https://secure.gravatar.com/avatar/428cf084b50...,https://secure.gravatar.com/avatar/428cf084b50...,https://secure.gravatar.com/avatar/428cf084b50...,T040HKJE3
2,U6YSPBJK0,012345vov,0,Asia/Kuwait,Arabia Standard Time,10800,0,0,0,0,...,gecadf1ec70f,012345vov,,https://secure.gravatar.com/avatar/ecadf1ec70f...,https://secure.gravatar.com/avatar/ecadf1ec70f...,https://secure.gravatar.com/avatar/ecadf1ec70f...,https://secure.gravatar.com/avatar/ecadf1ec70f...,https://secure.gravatar.com/avatar/ecadf1ec70f...,https://secure.gravatar.com/avatar/ecadf1ec70f...,T040HKJE3
3,U8YE1BVPS,04ok11,0,Europe/Helsinki,Eastern European Time,7200,0,0,0,0,...,gc1c71b3389d,,,https://secure.gravatar.com/avatar/c1c71b3389d...,https://secure.gravatar.com/avatar/c1c71b3389d...,https://secure.gravatar.com/avatar/c1c71b3389d...,https://secure.gravatar.com/avatar/c1c71b3389d...,https://secure.gravatar.com/avatar/c1c71b3389d...,https://secure.gravatar.com/avatar/c1c71b3389d...,T040HKJE3
4,U6U25UP96,0xdev,0,Europe/Minsk,Further-eastern European Time,10800,0,0,0,0,...,83d3b0c58991,Piashchynski,Valery,https://avatars.slack-edge.com/2017-12-03/2818...,https://avatars.slack-edge.com/2017-12-03/2818...,https://avatars.slack-edge.com/2017-12-03/2818...,https://avatars.slack-edge.com/2017-12-03/2818...,https://avatars.slack-edge.com/2017-12-03/2818...,https://avatars.slack-edge.com/2017-12-03/2818...,T040HKJE3


In [5]:
messages.head()

,channel,ts,type,text,user,thread_ts,parent_user_id,subtype,reply_count,unread_count,bot_id,username
0,bots,1502440131.035604,message,<@U1Z2QA4EM> как избавиться от рекурсии?,U1UMQM200,,,,,,,
1,bots,1502440131.035853,message,<@U1UMQM200>: избавиться от искушения - это ка...,U1Z2QA4EM,,,,,,,
2,bots,1476874935.003142,message,<@U1Z2QA4EM> в психотерапию умеешь?,U09JEC7V0,,,,,,,
3,bots,1476874936.003143,message,<@U09JEC7V0>: ох уж этот реверс в аметисты сос...,U1Z2QA4EM,,,,,,,
4,bots,1476874950.003144,message,<@U1Z2QA4EM> может ты у мамки психолог?,U065VP6F7,,,,,,,


## Майнинг юзернеймов из сообщений

1) Гипотеза: если юзер пишет сообщение, в котором упоминает github/kaggle/linkedin с юзернеймом, равным юзернейму в слаке, то это он

2) Ну и похожие (по левенштейну) юзернеймы тоже считаются

In [6]:
df = pd.read_sql_query(
    'select usr.id, usr.name, msg.channel, msg.ts, msg.text \
    from imported_messages as msg inner join imported_user_data as usr on msg.user = usr.id', engine)

2018-03-20 18:12:05,773 INFO sqlalchemy.engine.base.Engine select usr.id, usr.name, msg.channel, msg.ts, msg.text     from imported_messages as msg inner join imported_user_data as usr on msg.user = usr.id
2018-03-20 18:12:05,774 INFO sqlalchemy.engine.base.Engine ()


In [7]:
print(df.shape)
df.head()


(740471, 5)


,id,name,channel,ts,text
0,U1UMQM200,iggisv9t,bots,1502440131.035604,<@U1Z2QA4EM> как избавиться от рекурсии?
1,U1Z2QA4EM,markov-bot,bots,1502440131.035853,<@U1UMQM200>: избавиться от искушения - это ка...
2,U09JEC7V0,ssviridov,bots,1476874935.003142,<@U1Z2QA4EM> в психотерапию умеешь?
3,U1Z2QA4EM,markov-bot,bots,1476874936.003143,<@U09JEC7V0>: ох уж этот реверс в аметисты сос...
4,U065VP6F7,libfun,bots,1476874950.003144,<@U1Z2QA4EM> может ты у мамки психолог?


In [8]:
def get_levenstein(df, th):
    df_match = df[df.apply(lambda row: (editdistance.eval(row['name'], row['mention']) <= th and 
                           (len(row['name']) > th  * 2 + 2 or len(row['mention']) > th * 2 + 2)), axis=1)]
#     print(df_match.shape)
#     print(df_match[df_match['name'] != df_match['mention']][['name', 'mention']].head())
    return df_match[['id', 'name', 'mention', 'url']]
#     return df_match

## Достанем из гитхаба (github.com/username и username.github.io) 

In [9]:
github_regex = '<http[^>]*github.com/([^>/]+)'

df_github = df[df.text.str.contains(github_regex)]
df_github['mention'] = ''
df_github['url'] = ''
for index, row in df_github.iterrows():
    df_github.loc[index]['name'] = df_github.loc[index]['name'].lower()
    m = re.search(github_regex, row['text'])
    if m:
        df_github.loc[index]['mention'] = m.group(1).lower()
        df_github.loc[index]['url'] = 'https://github.com/' + m.group(1).lower()
        

# df_github_match = df_github[df_github.apply(lambda row: row['name'] == row['mention'], axis=1)]
# df_github_match.shape
github_match = pd.DataFrame(columns=['id', 'name', 'mention', 'url'])

for th in range(6):
    github_match = github_match.append(get_levenstein(df_github, th))
    
githubio_regex = '/(\w+).github.io'

df_githubio = df[df.text.str.contains(githubio_regex)]
df_githubio['mention'] = ''
df_githubio['url'] = ''
for index, row in df_githubio.iterrows():
    df_githubio.loc[index]['name'] = df_githubio.loc[index]['name'].lower()
    m = re.search(githubio_regex, row['text'])
    if m:
        df_githubio.loc[index]['mention'] = m.group(1).lower()
        df_githubio.loc[index]['url'] = 'https://github.com/' + m.group(1).lower()

for th in range(6):
    github_match = github_match.append(get_levenstein(df_githubio, th))

github_match = github_match.drop_duplicates()

print(github_match.shape)
github_match[github_match['name'] != github_match['mention']]

/usr/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  app.launch_new_instance()
/usr/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3.6/site-packages/ipykernel/__main__.py:23: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/usr/lib/python3.6/

(119, 4)


,id,name,mention,url
54493,U054CQTS7,1aguschin,aguschin,https://github.com/aguschin
107775,U4PKEBC03,kit_scribe,kit-scribe,https://github.com/kit-scribe
108265,U6MNDLDT7,philip-b,philip-bl,https://github.com/philip-bl
142333,U0D6W1WTW,alex_sosn,alexsosn,https://github.com/alexsosn
145337,U04DXFZ2G,fedor.chervinskii,fedor-chervinskii,https://github.com/fedor-chervinskii
161409,U1ZCENED7,romul,lromul,https://github.com/lromul
468091,U6L7YLWHY,shkarupa.alex,shkarupa-alex,https://github.com/shkarupa-alex
636832,U4RFX4PJT,nerses_bagiyan,nersesbagiyan,https://github.com/nersesbagiyan
636862,U04725QK7,gleb_filatov,glebfilatov,https://github.com/glebfilatov
636894,U2ZJ517B8,sharlottik,sharlotik,https://github.com/sharlotik


## Достанем из kaggle (kaggle.com/username) 

In [10]:
kaggle_regex = '<http[^>]*kaggle.com/([^>/]+)'

df_kaggle = df[df.text.str.contains(kaggle_regex)]
df_kaggle['mention'] = ''
df_kaggle['url'] = ''
for index, row in df_kaggle.iterrows():
    df_kaggle.loc[index]['name'] = df_kaggle.loc[index]['name'].lower()
    m = re.search(kaggle_regex, row['text'])
    if m:
        df_kaggle.loc[index]['mention'] = m.group(1).lower()
        df_kaggle.loc[index]['url'] = 'https://kaggle.com/' +  m.group(1).lower()
        

# df_kaggle_match = df_kaggle[df_kaggle.apply(lambda row: row['name'] == row['mention'], axis=1)]
# df_kaggle_match.shape
kaggle_match = pd.DataFrame(columns=['id', 'name', 'mention', 'url'])

for th in range(6):
    kaggle_match = kaggle_match.append(get_levenstein(df_kaggle, th))

kaggle_match = kaggle_match.drop_duplicates()

print(kaggle_match.shape)
kaggle_match[kaggle_match['name'] != kaggle_match['mention']]

/usr/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  app.launch_new_instance()
/usr/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(94, 4)


,id,name,mention,url
50333,U4FCBEVAS,inga.kaydanova,ingakaydanova,https://kaggle.com/ingakaydanova
50335,U4XSYA99C,andrey.,andreyg,https://kaggle.com/andreyg
50353,U2Z07JW4T,alex.radionov,alexradionov,https://kaggle.com/alexradionov
50372,U2T2WU8L9,fartuk,fartuk1,https://kaggle.com/fartuk1
50373,U4HS6PVBR,alelu,alexlu,https://kaggle.com/alexlu
50377,U4X5SNMK2,vfdev,vfdev5,https://kaggle.com/vfdev5
54783,U0H7VBQQ1,alex.ozerin,alexozerin,https://kaggle.com/alexozerin
58684,U1BKBLY4X,antter7,antter,https://kaggle.com/antter
61554,U86DVN6G1,pavelosta,pavelost,https://kaggle.com/pavelost
243970,U436BP343,waniz,wanizz,https://kaggle.com/wanizz


## Достанем из линкедина (linkedin.com/in/username)

Плохо работает, т.к. в линкедине часто name-surname-some_random_hash

In [11]:
linkedin_regex = '<http[^>]*linkedin.com/in/([^>/]+)'

df_linkedin = df[df.text.str.contains(linkedin_regex)]
df_linkedin['mention'] = ''
df_linkedin['url'] = ''
for index, row in df_linkedin.iterrows():
    df_linkedin.loc[index]['name'] = df_linkedin.loc[index]['name'].lower()
    m = re.search(linkedin_regex, row['text'])
    if m:
        df_linkedin.loc[index]['mention'] = m.group(1).lower()
        df_linkedin.loc[index]['url'] = 'https://www.linkedin.com/in/' + m.group(1).lower()
        

# df_linkedin_match = df_linkedin[df_linkedin.apply(lambda row: row['name'] == row['mention'], axis=1)]
# df_linkedin_match.shape
linkedin_match = pd.DataFrame(columns=['id', 'name', 'mention', 'url'])

for th in range(6):
    linkedin_match = linkedin_match.append(get_levenstein(df_linkedin, th))

linkedin_match = linkedin_match.drop_duplicates()

print(linkedin_match.shape)
linkedin_match[linkedin_match['name'] != linkedin_match['mention']]

/usr/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  app.launch_new_instance()


(30, 4)


/usr/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,id,name,mention,url
244076,U4GD9N3GB,valentin.mikhaylenko,valentinmikhaylenko,https://www.linkedin.com/in/valentinmikhaylenko
244081,U1NLYUY5N,leonid.danilchenko,leonid-danilchenko,https://www.linkedin.com/in/leonid-danilchenko
253293,U049HDR2Z,egor.panfilov,egorpanfilov,https://www.linkedin.com/in/egorpanfilov
253388,U0ZHCGY5T,egor_labintcev,egorlabintcev,https://www.linkedin.com/in/egorlabintcev
228321,U30Q72KLJ,tchernov,timchernov,https://www.linkedin.com/in/timchernov
253368,U30A6KZLH,atselikov,alextselikov,https://www.linkedin.com/in/alextselikov
548810,U3UB29PTJ,iryndin,ivanryndin,https://www.linkedin.com/in/ivanryndin
286736,U364MEXR9,a.kumpan,antonkumpan,https://www.linkedin.com/in/antonkumpan


In [12]:
best = github_match.merge(kaggle_match.merge(linkedin_match, how='inner', on=['name']), how='inner', on=['name'])
print(best)
best = github_match.merge(kaggle_match, how='inner', on=['name'])
print(best)

          id         name      mention                             url  \
0  U04CH4QBD  ermakovpetr  ermakovpetr  https://github.com/ermakovpetr   
1  U2LGQMC4D     venheads     venheads     https://github.com/venheads   

        id_x    mention_x                           url_x       id_y  \
0  U04CH4QBD  ermakovpetr  https://kaggle.com/ermakovpetr  U04CH4QBD   
1  U2LGQMC4D     venheads     https://kaggle.com/venheads  U2LGQMC4D   

     mention_y                                    url_y  
0  ermakovpetr  https://www.linkedin.com/in/ermakovpetr  
1     venheads     https://www.linkedin.com/in/venheads  
         id_x           name      mention_x                             url_x  \
0   U07V1URT9          n01z3          n01z3          https://github.com/n01z3   
1   U0FEJNBGQ         mabrek         mabrek         https://github.com/mabrek   
2   U0JHK9001        rushter        rushter        https://github.com/rushter   
3   U3PLVL3RD         jgc128         jgc128         https://gi

### Всё в 1 датафрейм

In [13]:
# total = pd.DataFrame(columns=['id', 'name', 'key', 'value'])

# github_match['key'] = 'github'
# kaggle_match['key'] = 'kaggle'
# linkedin_match['key'] = 'linkedin'

# github_match = github_match.rename(columns={"mention": "value"})
# kaggle_match = kaggle_match.rename(columns={"mention": "value"})
# linkedin_match = linkedin_match.rename(columns={"mention": "value"})

# print(kaggle_match.head())
# total = github_match.merge(kaggle_match.merge(linkedin_match, how='outer', on=['id', 'name', 'key', 'value']), how='outer', on=['id', 'name', 'key', 'value'])
# print(total.shape)
# print(total.head())
# total[['id', 'key', 'value']].to_csv('user_profiles.csv', index=False)

              id         name        value                             url  \
45085  U43FTJQ2V      zfturbo      zfturbo      https://kaggle.com/zfturbo   
46996  U6GTG35V0  andrew32bit  andrew32bit  https://kaggle.com/andrew32bit   
50331  U1V57JCUF       tenich       tenich       https://kaggle.com/tenich   
50334  U4BBNMQ03      justart      justart      https://kaggle.com/justart   
50341  U4D7EK3N3     funkindy     funkindy     https://kaggle.com/funkindy   

          key  
45085  kaggle  
46996  kaggle  
50331  kaggle  
50334  kaggle  
50341  kaggle  
(243, 7)
          id     name    value                         url     key url_x url_y
0  U07V1URT9    n01z3    n01z3    https://github.com/n01z3  github   NaN   NaN
1  U0FEJNBGQ   mabrek   mabrek   https://github.com/mabrek  github   NaN   NaN
2  U0JHK9001  rushter  rushter  https://github.com/rushter  github   NaN   NaN
3  U3PLVL3RD   jgc128   jgc128   https://github.com/jgc128  github   NaN   NaN
4  U4P3L4R19    sh1ng    sh1ng 

In [14]:
total

,id,name,value,url,key,url_x,url_y
0,U07V1URT9,n01z3,n01z3,https://github.com/n01z3,github,NaN,NaN
1,U0FEJNBGQ,mabrek,mabrek,https://github.com/mabrek,github,NaN,NaN
2,U0JHK9001,rushter,rushter,https://github.com/rushter,github,NaN,NaN
3,U3PLVL3RD,jgc128,jgc128,https://github.com/jgc128,github,NaN,NaN
4,U4P3L4R19,sh1ng,sh1ng,https://github.com/sh1ng,github,NaN,NaN
5,U0ZJV6E5Q,movchan74,movchan74,https://github.com/movchan74,github,NaN,NaN
6,U8H6V2ME3,malyutins,malyutins,https://github.com/malyutins,github,NaN,NaN
7,U06J1LG1M,mephistopheies,mephistopheies,https://github.com/mephistopheies,github,NaN,NaN
8,U2ZHRCAUB,generall,generall,https://github.com/generall,github,NaN,NaN
9,U043814R6,lextal,lextal,https://github.com/lextal,github,NaN,NaN


In [15]:
fields = ['id', 'name', 'title', 'real_name_normalized', 'first_name', 'last_name', 'skype', 'github', 'url']

fields_ods = {'skype' : 'Xf0DANL9SL', 'github' : 'Xf3WC3HJMR' }
def parse_messages_get_fields(json_path):
    data = json.load(open(json_path))
    rows_list = []
    with open('fields.csv', 'w') as f:
        for user in data:
            skype = ''
            github = ''
            if user['profile'].get('fields', ''):
                if fields_ods['skype'] in user['profile']['fields']:
                    skype = user['profile']['fields'][fields_ods['skype']]['value']
#                     if skype['alt'] != '':
#                         skype = skype['alt']
#                     else:
#                         skype = skype['value']
                if fields_ods['github'] in user['profile']['fields']:
                    github = user['profile']['fields'][fields_ods['github']]['value']
#                     if github['alt'] != '':
#                         github = github['alt']
#                     else:
#                         github = github['value']
            dict1 = {'id' : user['id'],
                     'name' : user['name'], 
                     'title' : user['profile']['title'],
                     'real_name_normalized' : user['profile'].get('real_name_normalized', ''),
                     'first_name' : user['profile'].get('first_name', ''),
                     'last_name' : user['profile'].get('last_name', ''),
                     'skype' : skype,
                     'github' : github, 'url' : github}
            rows_list.append(dict1)
    return pd.DataFrame(rows_list)      

df_users = parse_messages_get_fields('../data/users.json')
df_users.head()

,first_name,github,id,last_name,name,real_name_normalized,skype,title,url
0,0,,U4EMUGFNG,,0,0,,Just a beginner. Learning Python. Если нужен п...,
1,,,U7SE7ANCW,,00serd00,Sergey Vdovin,,,
2,012345vov,,U6YSPBJK0,,012345vov,012345vov,,,
3,,,U8YE1BVPS,,04ok11,04ok11,,,
4,Piashchynski,,U6U25UP96,Valery,0xdev,Piashchynski Valery,xdevbel,,


In [16]:
df_users_github_match = df_users[df_users.apply(lambda row: type(row['github']) == type('a') and
                                 len(str(row['github'] )) > 0, axis=1)][['id', 'name', 'github']]


In [17]:
total = pd.DataFrame(columns=['id', 'name', 'key', 'value'])

github_match['key'] = 'github'
kaggle_match['key'] = 'kaggle'
linkedin_match['key'] = 'linkedin'
df_users_github_match['key'] = 'github'
for index, row in df_users_github_match.iterrows():
    if 'kaggle' in row['github']:
        row['key'] = 'kaggle'
    if 'linkedin' in row['github']:
        row['key'] = 'linkedin'

print(df_users_github_match[df_users_github_match['key'] == 'kaggle'])

print(github_match.columns)
print(kaggle_match.columns)
print(linkedin_match.columns)

github_match = github_match.rename(columns={"url": "value"})
kaggle_match = kaggle_match.rename(columns={"url": "value"})
linkedin_match = linkedin_match.rename(columns={"url": "value"})
df_users_github_match = df_users_github_match.rename(columns={"github": "value"})

# print(kaggle_match.head())
print(github_match.columns)
print(kaggle_match.columns)
print(linkedin_match.columns)
total = kaggle_match[['id', 'name', 'key', 'value']].merge(linkedin_match[['id', 'name', 'key', 'value']], how='outer', on=['id', 'name', 'key', 'value'])
total = total.merge(github_match[['id', 'name', 'key', 'value']], how='outer', on=['id', 'name', 'key', 'value'])
total = total.merge(df_users_github_match, how='outer', on=['id', 'name', 'key', 'value'])


              id       name                           github     key
7768   U86DVN6G1  pavelosta  https://www.kaggle.com/pavelost  kaggle
11061  U43FTJQ2V    zfturbo   https://www.kaggle.com/zfturbo  kaggle
Index(['id', 'name', 'value', 'url', 'key'], dtype='object')
Index(['id', 'name', 'value', 'url', 'key'], dtype='object')
Index(['id', 'name', 'value', 'url', 'key'], dtype='object')
Index(['id', 'name', 'value', 'value', 'key'], dtype='object')
Index(['id', 'name', 'value', 'value', 'key'], dtype='object')
Index(['id', 'name', 'value', 'value', 'key'], dtype='object')


ValueError: Buffer has wrong number of dimensions (expected 1, got 2)

## Запись результатов в csv

In [ ]:
total = total.drop_duplicates(['id', 'key'])
print(total.shape)
print(total.head())
total[['id', 'key', 'value']].to_csv('user_profiles.csv', index=False)